# Week 5 Final Project

## 1. Import libraries and set global variables required for the rest of the project.

Import libraries

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy import distance

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import branca
import branca.colormap as cm

print('Libraries imported.')

CLIENT_ID = 'MXIL5MGSP25GK0AHGJZIKWMFDN5F43OFVEZLBLGKRYHBAD2A' # your Foursquare ID
CLIENT_SECRET = '2H5DFT4NQIODSIG2TT5YJV1RDKGEIPJL2S23W5DFBL1ZH3XX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius=500

Libraries imported.


## 2. Get GeoCodes for Toronto and Montreal

In [ ]:
geocodes={'Toronto':[0,0], 'Montreal':[0,0]}

address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
geocodes['Toronto'][0] = location.latitude
geocodes ['Toronto'][1]= location.longitude

address = 'Montreal, Quebec, Canada'
location = geolocator.geocode(address)
geocodes['Montreal'][0] = location.latitude
geocodes ['Montreal'][1]= location.longitude

In [ ]:
geocodes

{'Montreal': [45.4972159, -73.6103642], 'Toronto': [43.6534817, -79.3839347]}

## 3. Get venues for both Toronto and Montreal using the Foursquare APIs

function to get venues (limited to 100 venues per city due to Foursqaure API constraints for free account)

In [ ]:
def getNearbyVenues(latitude, longitude, radius=5000):
    
    venues_list=[]
            
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          radius, 
          LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call function to get venues for Toronto

In [ ]:
toronto_venues = getNearbyVenues(geocodes['Toronto'][0], geocodes['Toronto'][1])

Call function to get venues for Montreal

In [ ]:
montreal_venues = getNearbyVenues(geocodes['Montreal'][0], geocodes['Montreal'][1])

## 4. Get the median distance for each venue from other Pizza Places

Define function that will iterate through the venues and calculate the median distance between each venue and other Pizza places. Distance is calculated using geo coordinates

In [ ]:
def getMedianDistancefromPizzaPlaces (df_venues):
  df_iteration = df_venues.copy()
  df_pizza_places = df_iteration[df_iteration['Venue Category'].str.contains("Pizza")]
  for index_1, row_1 in df_iteration.iterrows():

    location_gps_1 = (row_1['Venue Latitude'], row_1['Venue Longitude'])
    
    for index, row in df_pizza_places.iterrows():
      location_gps_2 = (row['Venue Latitude'], row['Venue Longitude'])
      df_iteration.at[index_1,'PizzaDistance'] = distance.distance(location_gps_1, location_gps_2).miles
    df_venues.at[index_1,'PizzaDistance'] = df_iteration['PizzaDistance'].median()

Call function to update the median distance for each venue in Toronto from other Pizza places.

In [ ]:
getMedianDistancefromPizzaPlaces(toronto_venues)
toronto_venues['PizzaDistance'].describe()

count    100.000000
mean       0.915999
std        0.049071
min        0.668078
25%        0.892680
50%        0.919178
75%        0.951509
max        0.980175
Name: PizzaDistance, dtype: float64

Call function to update the median distance for each venue in Montreal from other Pizza places.

In [ ]:
getMedianDistancefromPizzaPlaces(montreal_venues)
montreal_venues['PizzaDistance'].describe()

count    100.000000
mean       1.803250
std        0.315099
min        1.492180
25%        1.542347
50%        1.581167
75%        1.997232
max        2.348751
Name: PizzaDistance, dtype: float64

## 5. Create Maps with gradient color markers based on the median distance

Bluish colors shows venues where Pizza Places are further away and hence would be good places for the new concept parlors

Create the map for Toronto

In [ ]:
# create map of Manhattan using latitude and longitude values
map = folium.Map(location=[geocodes['Toronto'][0], geocodes['Toronto'][1]], zoom_start=13)

colormap = cm.LinearColormap(colors=['red', 'blue'], vmin=toronto_venues['PizzaDistance'].min(), vmax=toronto_venues['PizzaDistance'].max())
colormap.caption = "Distance from Pizza Places (miles)"
map.add_child(colormap)

# add markers to map
for lat, lng, label, p in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['PizzaDistance']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colormap(p),
        fill=True,
        fill_color=colormap(p),
        fill_opacity=1.0,
        parse_html=False).add_to(map) 
map

Create the map for Montreal

In [ ]:

# create map of Manhattan using latitude and longitude values
map = folium.Map(location=[geocodes['Montreal'][0], geocodes['Montreal'][1]], zoom_start=13)

colormap = cm.LinearColormap(colors=['red', 'blue'], vmin=montreal_venues['PizzaDistance'].min(), vmax=montreal_venues['PizzaDistance'].max())
colormap.caption = "Distance from Pizza Places (miles)"
map.add_child(colormap)

# add markers to map
for lat, lng, label, p in zip(montreal_venues['Venue Latitude'], montreal_venues['Venue Longitude'], montreal_venues['Venue'], montreal_venues['PizzaDistance']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colormap(p),
        fill=True,
        fill_color=colormap(p),
        fill_opacity=1.0,
        parse_html=False).add_to(map)  
map